### Experiment: Single Objective Variance of True Probablity

#### Aim

Each row is provided as marks which will be allotted to the **individual** if it solves that row. Each row has marks = **pop_size**.

If one individual solves a problem then the marks allotted to that row will decrease so that other inidividuals can focus on other rows for gaining marks. But the marks will not decresase beyond **40% of pop_size**

My only concern is that in last when it becomes an ensemble will there be enough votes for the ensemble to predict?

#### How does it make sure the accuracy is also increasing?


#### Critism

In [1]:
import operator
import itertools
import numpy as np
import seaborn as sb

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split

import matplotlib.pyplot as plt

np.seterr(all='raise')

digits = load_digits()
digit_features, digit_labels = digits.data, digits.target
X_train, X_test, y_train, y_test = train_test_split(digit_features, digit_labels, stratify=digit_labels,train_size=0.75, test_size=0.25)

In [2]:
# The exploration of the dataset by benchmark algorithms
clf = DecisionTreeClassifier(random_state=34092)
clf.fit(X_train, y_train)
pred_DTC = clf.predict(X_test)
print('Base DecisionTreeClassifier accuracy: {}'.format(clf.score(X_test, y_test)))

clf = RandomForestClassifier(random_state=34092)
clf.fit(X_train, y_train)
pred_RFC = clf.predict(X_test)
print('Base RandomForestClassifier accuracy: {}'.format(clf.score(X_test, y_test)))

clf = GradientBoostingClassifier(random_state=34092)
clf.fit(X_train, y_train)
pred_GBC = clf.predict(X_test)
print('Base GradientBoostingClassifier accuracy: {}'.format(clf.score(X_test, y_test)))

print('')

Base DecisionTreeClassifier accuracy: 0.851441241685
Base RandomForestClassifier accuracy: 0.942350332594
Base GradientBoostingClassifier accuracy: 0.955654101996



In [3]:
max_marks = 100
min_marks = 0.33*max_marks
diff_global_marks = np.ones(y_test.shape[0])
diff_global_marks = diff_global_marks*max_marks

In [4]:
def predict_unseen(population):
    forest_predictions = []
    for ind_num, individual in enumerate(population):
        func = toolbox.compile(expr=individual)
        
        
        sample_counts = [int(func(*record)) for record in X_train]
        sample_counts = [max(min(sample_count, 10), 0) for sample_count in sample_counts]
        sample = []
        for sample_index, sample_count in enumerate(sample_counts):
            sample.extend([sample_index] * sample_count)
        sample = np.array(sample)

        if len(sample) == 0:
            return 1e-20, 1e-20

        clf = DecisionTreeClassifier(random_state=34092)
        clf.fit(X_train[sample], y_train[sample])
        predictions = clf.predict(X_test)
        forest_predictions.append(predictions)
        
    from collections import Counter
    from sklearn.metrics import accuracy_score
    
    y_pred = np.array([Counter(instance_forest_predictions).most_common(1)[0][0] for instance_forest_predictions in zip(*forest_predictions)])
    #np.sum(y_test == y_pred) / len(y_test)
    print "Accuracy->"+str(np.sum(y_test == y_pred)*100/ len(y_test))

In [5]:
# defined a new primitive set for strongly typed GP
pset = gp.PrimitiveSetTyped('MAIN', itertools.repeat(float, digit_features.shape[1]), bool, 'Feature')

# boolean operators
pset.addPrimitive(operator.and_, [bool, bool], bool)
pset.addPrimitive(operator.or_, [bool, bool], bool)
pset.addPrimitive(operator.not_, [bool], bool)

# floating point operators
# Define a protected division function
def protectedDiv(left, right):
    try: return left / right
    except (ZeroDivisionError, FloatingPointError): return 1.

pset.addPrimitive(operator.add, [float, float], float)
pset.addPrimitive(operator.sub, [float, float], float)
pset.addPrimitive(operator.mul, [float, float], float)
pset.addPrimitive(protectedDiv, [float, float], float)

# logic operators
# Define a new if-then-else function
def if_then_else(in1, output1, output2):
    if in1: return output1
    else: return output2

pset.addPrimitive(operator.lt, [float, float], bool)
pset.addPrimitive(operator.eq, [float, float], bool)
pset.addPrimitive(if_then_else, [bool, float, float], float)

# terminals
pset.addTerminal(False, bool)
pset.addTerminal(True, bool)
for val in np.arange(-10., 11.):
    pset.addTerminal(val, float)

creator.create('FitnessMax', base.Fitness, weights=(1.0, 1.0))
creator.create('Individual', gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('expr', gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('compile', gp.compile, pset=pset)

In [6]:
def evaluate_individual(individual):
    global diff_global_marks
    # Transform the tree expression into a callable function
    func = toolbox.compile(expr=individual)
    
    sample_counts = [int(func(*record)) for record in X_train]
    sample_counts = [max(min(sample_count, 10), 0) for sample_count in sample_counts]
    sample = []
    for sample_index, sample_count in enumerate(sample_counts):
        sample.extend([sample_index] * sample_count)
    sample = np.array(sample)
    
    if len(sample) == 0:
        return 1e-20, 1e-20
    
    clf = DecisionTreeClassifier(random_state=34092)
    clf.fit(X_train[sample], y_train[sample])
    #score = clf.score(X_test, y_test)
    t_pred = clf.predict(X_test)
    total_marks = np.sum((t_pred==y_test)*diff_global_marks)
    accuracy = np.sum((t_pred==y_test))
    
    # Updating the diff_marks
    bool_ = 1*(t_pred==y_test)
    diff_global_marks = diff_global_marks-bool_
    diff_global_marks[diff_global_marks<min_marks] = min_marks
    
    return total_marks, accuracy

In [7]:
toolbox.register('evaluate', evaluate_individual)
#toolbox.register('select', tools.selTournament, tournsize=3)
toolbox.register("select", tools.selNSGA2)
toolbox.register('mate', gp.cxOnePoint)
toolbox.register('expr_mut', gp.genFull, min_=0, max_=3)
toolbox.register('mutate', gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

population = toolbox.population(n=max_marks)
halloffame = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register('std', np.std)
stats.register('min', np.min)
stats.register('avg', np.mean)
stats.register('max', np.max)

cxpb = 0.5
mutpb = 0.5
lambda_ = 100
mu = max_marks
ngen = 100
verbose = True

logbook = tools.Logbook()
logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])


# Evaluate the individuals with an invalid fitness
invalid_ind = [ind for ind in population if not ind.fitness.valid]
fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
for ind, fit in zip(invalid_ind, fitnesses):
    ind.fitness.values = fit

if halloffame is not None:
    halloffame.update(population)

logbook = tools.Logbook()
logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

record = stats.compile(population) if stats is not None else {}
logbook.record(gen=0, nevals=len(invalid_ind), **record)
if verbose:
    print logbook.stream

# Begin the generational process
global diff_global_marks
all_marks = []
all_marks.append(diff_global_marks)

for gen in range(1, ngen + 1):
    # Vary the population
    offspring = algorithms.varOr(population, toolbox, lambda_, cxpb, mutpb)

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # Update the hall of fame with the generated individuals
    if halloffame is not None:
        halloffame.update(offspring)

    # Select the next generation population
    population[:] = toolbox.select(offspring, mu)
    print len(population)
    predict_unseen(population)
    
    # Just updating the all_marks array which has all the changed values
    all_marks.append(diff_global_marks)
    
    # Update the statistics with the new population
    record = stats.compile(population) if stats is not None else {}
    logbook.record(gen=gen, nevals=len(invalid_ind), **record)
    if verbose:
        print logbook.stream
    #return population, logbook

str(halloffame[0])

gen	nevals	std    	min  	avg   	max  
0  	100   	11686.3	1e-20	7058.3	38400
100
1  	100   	5909.49	1e-20	3758.2	21456
100
2  	100   	4637.75	1e-20	2871.73	13412
100
3  	100   	4454.48	1e-20	2662.49	13073
100
4  	100   	4681.23	1e-20	2990.58	12852
100
5  	100   	4927.7 	1e-20	3415.08	12804
100
6  	100   	4547.75	1e-20	2993.55	12827
100
7  	100   	4780.37	1e-20	3421.61	12936
100
8  	100   	4438.07	1e-20	3068.36	12672
100
9  	100   	4622.99	1e-20	3511.12	12706
100
10 	100   	4603.29	1e-20	3633.3 	12672
100
11 	100   	4398.78	1e-20	3175.49	12705
100
12 	100   	4524.34	1e-20	3336.46	12739
100
13 	100   	4743.11	1e-20	3689.97	12672
100
14 	100   	4801.58	1e-20	3684.53	12804
100
15 	100   	4918.7 	1e-20	4025.07	12738
100
16 	100   	4685.65	1e-20	3496.32	12771
100
17 	100   	4831.21	1e-20	3540.59	12903
100
18 	100   	4799.08	1e-20	3655.85	12903
100
19 	100   	5089.86	1e-20	3801.03	12903
100
20 	100   	5215.95	1e-20	4004.35	12804
100
21 	100   	5074.6 	1e-20	3633.58	12804
100
22 	100   	5381.54

SyntaxError: 'return' outside function (<ipython-input-7-75859c173a70>, line 76)

NameError: name 'sample' is not defined

In [ ]:
marks = pd.DataFrame(all_marks[0])

In [ ]:
marks.columns = [str(1)]

In [ ]:
marks_ = pd.DataFrame()
for i in range(0,len(all_marks)):
    if(i==0):
        marks_ = pd.DataFrame(all_marks[0])
    else:
        if(type(all_marks[i])!=float):
            temp_ = pd.DataFrame(all_marks[i])
            temp_.columns = [i]
            marks_ = pd.concat([marks_,temp_],axis=1)

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(marks_[0])

In [ ]:
forest_predictions = []

for ind_num, individual in enumerate(population):
    func = toolbox.compile(expr=individual)
    subsample = np.array([func(*record) for record in X_train])
    
    if X_train[subsample].shape[0] == 0:
        continue
    
    clf = DecisionTreeClassifier(random_state=34092)
    clf.fit(X_train[subsample], y_train[subsample])
    predictions = clf.predict(X_test)
    forest_predictions.append(predictions)

In [ ]:
from collections import Counter
from sklearn.metrics import accuracy_score

y_pred = np.array(
    [Counter(instance_forest_predictions).most_common(1)[0][0] for instance_forest_predictions in zip(*forest_predictions)])
#np.sum(y_test == y_pred) / len(y_test)
np.sum(y_test == y_pred)*100/ len(y_test)

In [ ]:
func = toolbox.compile(expr=halloffame[0])
subsample = np.array([func(*record) for record in X_train])

In [ ]:
subsample

In [ ]:
print halloffame[0]

In [22]:
chk

1

In [15]:
(0.5*100)

SyntaxError: unexpected EOF while parsing (<ipython-input-15-db75ffb14134>, line 1)

In [21]:
chk = 1;
